In [1]:
from pathlib import Path
import pandas as pd, numpy as np
import shutil, json, yaml

In [2]:
base_folder = Path("/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/")
result_folder = Path("/media/filer2/T4b/Birds/Songs")

In [3]:
files = list(base_folder.glob("**/*"))
files

[PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/B60'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/Bird47'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/BlackCyan'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/Green23'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/GreenGreen'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/Red2'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/RedRed'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/B60/GlobUnitsStatsOld'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/B60/README.txt'),
 PosixPath('/media/filer2/T4b/Carmen/LMANX_corr

In [4]:
df = pd.DataFrame()
df["song_npy"] = [f for f in files if f.parent.stem=="song" and f.suffix==".npy"]
df["bird"] = df["song_npy"] .apply(lambda f: f.relative_to(base_folder).parts[0])
df["is_unused"] = df["song_npy"] .apply(lambda f: "unused" in str(f).lower())
def get_date(name):
    import re
    patterns = [
        '(?P<day>\d{2})_(?P<month>\d{2})_(?P<year>\d{4})_', 
        '(?P<year>\d{4})_(?P<month>\d{2})_(?P<day>\d{2})_',
        '(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})_',
        '(?P<year>\d{4})_(?P<month>\d{2})(?P<day>\d{2})_',
        '^(?P<month>\d{1})-(?P<day>\d{2})-(?P<year>\d{2})_'
    ]
    d = {}
    prev_pattern = None
    for p in patterns:
        m = list(re.finditer(p, name))
        if len(m) ==0:
            continue
        elif len(m) > 1:
            raise Exception("several matches in pattern")
        elif prev_pattern is not None:
            raise Exception(f"several match pattern {p} {prev_pattern} for {name}")
        else:
            prev_pattern = p
            d["day"] = m[0]["day"]
            d["month"] = m[0]["month"] if len(m[0]["month"]) ==2 else "0" + m[0]["month"]
            d["year"] = m[0]["year"] if len(m[0]["year"]) ==4 else "20" + m[0]["year"]
    if len(d) > 0:
        return f'{d["year"]}-{d["month"]}-{d["day"]}'
    else: 
        last = int(name[-4:])
        if last < 41:
            return "2014-01-13"
        elif last < 69:
            return "2014-01-14"
        elif last< 79:
            return "2014-01-15"
        else:
            return "2014-01-16"
df["date"] =  df["song_npy"] .apply(lambda f: get_date(f.parent.parent.stem))
df.groupby(["bird", "date", "is_unused"], dropna=False).count()

song_npy
bird       date       is_unused          
B60        2022-04-19 False             2
           2022-04-20 False             4
           2022-04-21 False             1
           2022-04-22 False             2
           2022-04-23 False             1
           2022-04-25 False             3
           2022-04-26 False             4
           2022-04-28 False             3
           2022-04-29 False             3
           2022-04-30 False             2
           2022-05-04 False             3
           2022-05-05 False             4
           2022-05-06 False             3
           2022-05-07 False             1
Bird47     2021-11-22 False             3
           2021-11-24 False             3
           2021-11-25 False             2
           2021-11-26 False             3
BlackCyan  2021-11-18 True              1
           2021-11-19 False             2
                      True              1
Green23    2015-03-03 False             2
           2015-03-04 False             3
           2015-03-05 False             1
           2015-03-06 False             4
           2015-03-09 False             4
           2015-03-10 False             5
           2015-03-12 False             8
           2015-03-13 False             2
GreenGreen 2015-01-29 False             7
Red2       2014-01-13 False            14
           2014-01-14 False             3
RedRed     2022-03-11 False             2
           2022-03-13 False            68
                      True             68

In [5]:
def load_fs(f):
    meta = f.parent.parent/"metadata.json"
    if meta.exists():
        with meta.open("r") as s:
            try:
                d =  json.load(s)
            except:
                return np.nan
        if "fs_mic" in d:
            return d["fs_mic"]
    return np.nan
df["fs"] = df["song_npy"].apply(load_fs)
df.groupby(["bird", "fs", "is_unused"], dropna=False)["song_npy"].count()

bird        fs            is_unused
B60         32000.000000  False        26
                          False        10
Bird47      32000.000000  False         6
                          False         5
BlackCyan   10593.407113  False         2
                          True          2
Green23     32000.000000  False        29
GreenGreen  32000.000000  False         7
Red2        22321.428299  False        17
RedRed      32723.037368  False        70
            NaN           True         68
Name: song_npy, dtype: int64

In [6]:
filtered_df = df[df["fs"].notnull() & ~df["is_unused"]].copy()
filtered_df["dest_path"] = str(result_folder) + "/" + filtered_df["bird"] + "/" + filtered_df["date"] + "/" + filtered_df["song_npy"].apply(lambda f: (str(f.parent.parent.name) + "__" + str(f.stem)).replace(" ", "_")) + ".wav"
if filtered_df["dest_path"].duplicated().any():
    print(filtered_df[filtered_df["dest_path"].duplicated(keep=False)].to_string())
    raise Exception(f'duplication...')
    
print(filtered_df.to_string())

                                                                                                                                                             song_npy        bird  is_unused        date            fs                                                                                                    dest_path
0                         /media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/B60/Sessions/2022-04-19/2022-04-19_11-03-08/song/CSC20.npy         B60      False  2022-04-19  32000.000000                                  /media/filer2/T4b/Birds/Songs/B60/2022-04-19/2022-04-19_11-03-08__CSC20.wav
1                         /media/filer2/T4b/Carmen/LMANX_correlations_project/LMANX_behavior_data/BirdData/B60/Sessions/2022-04-19/2022-04-19_20-23-53/song/CSC20.npy         B60      False  2022-04-19  32000.000000                                  /media/filer2/T4b/Birds/Songs/B60/2022-04-19/2022-04-19_20-23-53__CSC20.wav
2                         /m

In [7]:

from tqdm.auto import tqdm
for _, row in tqdm(filtered_df.iterrows(), total=len(filtered_df.index)):
    npy, fs, dest = row["song_npy"], row["fs"], Path(row["dest_path"])
    if dest.exists(): continue
    try:
        data = np.load(npy).reshape(-1)
        if data.size > 10**8:
            print(f"Error with file {npy}. Exception file is too big")
            continue
        import scipy.io, scipy.signal
        if fs > 20000:
            filter = scipy.signal.butter(3, [100, 10000], btype="bandpass", output="sos", fs=fs)
        else:
            filter = scipy.signal.butter(3, 100, btype="highpass", output="sos", fs=fs)
        filtered_data = scipy.signal.sosfiltfilt(filter, data)
        if fs - int(fs) > 0.0000001:
            aligned_filtered_data = np.interp(np.arange(filtered_data.size)/np.round(fs), np.arange(filtered_data.size)/fs, filtered_data)
            new_fs = int(np.round(fs))
        else:
            aligned_filtered_data = filtered_data
            new_fs = int(fs)
        dest.parent.mkdir(exist_ok=True, parents=True)
        tmp_dest = dest.with_stem(dest.with_suffix(".wav.tmp"))
        scipy.io.wavfile.write(tmp_dest, new_fs, aligned_filtered_data)
        shutil.move(tmp_dest, dest)
    except Exception as e:
        print(f"Error with file {npy}. Exception {e}")


  0%|          | 0/172 [00:00<?, ?it/s]

: 